✍ Для начала давайте поближе познакомимся с данными в таблицах. Напишите запросы, которые позволят ответить на вопросы ниже. В поле для ввода напишите число, которое является ответом на вопрос, а в ноутбук-решение запишите сами запросы.

In [1]:
import pandas as pd
import psycopg2

In [2]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

connection = psycopg2.connect(
   dbname=DBNAME,
   user=USER,
   host=HOST,
   password=PASSWORD,
   port=PORT
)

Задание 3.1

Сколько вакансий есть в базе?

In [29]:
# код запроса представляет из себя строковую переменную
query_empl = f'''select 
                count(id)
           from public.vacancies
        '''

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
vacancies = pd.read_sql_query(query_empl, connection)
vacancies

,count
0,49197


Всего вакансий на момент исследования в базе находится 49197 вакансий, что вполне приличное количество для IT профессий

Задание 3.2

Сколько работодателей?

In [27]:
# код запроса представляет из себя строковую переменную
query_empl = f'''select 
                count(id)
           from public.employers
        '''

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
employers = pd.read_sql_query(query_empl, connection)
employers

,count
0,23501


Всего работодателей на момент исследования в базе находится 23501, что более чем в два раза меньше, чем вакансий. Значит в среднем каждой фирме требуется два и более специалистов

Задание 3.3

Сколько регионов?

In [28]:
# код запроса представляет из себя строковую переменную
query_industr = f'''select 
                        count(id)
           from public.areas
        '''

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
areas = pd.read_sql_query(query_industr, connection)
areas

,count
0,1362


Согласно данным из интернета в России в 22 году было 1118 городов. Исходя из количества регионов в базе IT специалисты уже требуются не только в города но и в меньшие населённые пункты. 

Задание 3.4

Сколько сфер деятельности?

In [25]:
# подсчёт сфер деятельности
query_industr = f'''select 
                        count(id)
           from public.industries
        '''

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
industries = pd.read_sql_query(query_industr, connection)
industries

,count
0,294


In [35]:
# Поиск самых популярных сфер деятельности
query_industr = f'''select 
                        i.name,
                        count(v.id)
           from public.industries i
           left join employers_industries ei on ei.industry_id = i.id
           left join vacancies v on ei.employer_id = v.employer_id
           group by i.name
           order by count(v.id) desc
        '''

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
industries_popular = pd.read_sql_query(query_industr, connection)
industries_popular.head(7)

,name,count
0,Разработка программного обеспечения,12499
1,"Системная интеграция, автоматизации технологи...",11034
2,"Интернет-компания (поисковики, платежные систе...",6413
3,Банк,2742
4,Мобильная связь,1585
5,Кадровые агентства,1143
6,Консалтинговые услуги,1137


Согласно дополнительному исследованию из 294 сфер деятельности в лидерах по востребованности являются разработка и внедрение програмного обеспечения - области деятельности связанные с созданием самих приложений и доведения их до пользователей.

Задание 4.1

Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).

Выберите пятёрку лидеров по количеству вакансий:

In [7]:
# код запроса представляет из себя строковую переменную
query_vac_are = f'''select 
                        a.name регион,
                        count(v.id) cnt
            from public.vacancies v
            left join public.areas a on v.area_id = a.id
            group by a.name
            order by cnt desc
        '''
# лучше использовать f-строки — так удобнее передавать параметры

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
vacanc_areas = pd.read_sql_query(query_vac_are, connection)
vacanc_areas.head(5)

,регион,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


В столице самое большое количество вакансий, что говорит о динамически развивающемся городе и частично большей плотности населения. 

Задание 4.2

Посмотрим на зарплаты. У какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой?

In [40]:
# количества вакансий с заполненным хотя бы одним из двух полей с зарплатой
query_vac_are = f'''select 
                        count(*) salary_from_or_to                      
            from public.vacancies
            where salary_from is NOT NULL or salary_to is NOT NULL
        '''
# лучше использовать f-строки — так удобнее передавать параметры

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
vacanc_employer = pd.read_sql_query(query_vac_are, connection)
vacanc_employer

,salary_from_or_to
0,24073


In [70]:
# Исследование на количество вакансий с границами зарплаты
query_vac_are = f'''
(    -- вакансии с верхним или нижнем уровенем зарплат
    select 
    'или нижняя или верхняя' границы_зарплаты,
        count(*) зарплата                    
    from public.vacancies
    where salary_from is NOT NULL or salary_to is NOT NULL
    
    union
    -- вакансии с верхним или нижнем уровенем зарплат
    select 
        'нижня и верхняя' ,
        count(*)                      
    from public.vacancies
    where salary_from is NOT NULL and salary_to is NOT NULL
    
    union
    -- вакансии с нижним уровенем зарплат
    select 
        'нижняя',
        count(*)                       
    from public.vacancies
    where salary_from is NOT NULL
    
    union
    -- вакансии с верхним уровенем зарплат
    select 
        'верхняя',
        count(*) salary_from                        
    from public.vacancies
    where salary_to is NOT NULL)
    
    order by зарплата

    '''
vacanc_employer = pd.read_sql_query(query_vac_are, connection)
vacanc_employer  


,границы_зарплаты,зарплата
0,нижня и верхняя,10102
1,верхняя,12690
2,нижняя,21485
3,или нижняя или верхняя,24073


В ходе дополнительного исследования по уровням зарплат выяснилось:
1. колличество вакансий в которых жёстко ограничены уровни зарплат более чем в два раза меньше, чем тех в которых не стит жесткого ограния.
2. количество вакансий с высоким карьерным ростом почти в два раза больше, чем тех где верхний уровень строго ограничен

Задание 4.3

Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого числа.

In [22]:
# код запроса представляет из себя строковую переменную
query_vac_are = f'''select 
                        round(avg(salary_to), 0) верхняя_граница,
                        round(avg(salary_from), 0) нижняя_граница
                        
            from public.vacancies
        '''
# лучше использовать f-строки — так удобнее передавать параметры

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
vacanc_employer = pd.read_sql_query(query_vac_are, connection)
vacanc_employer

,верхняя_граница,нижняя_граница
0,110537.0,71065.0


Хотя средняя граница по уровню зарплат - это как средняя температура по больнице, но и этот показатель показывает возможность карьернго роста почти в два раза

Задание 4.4

Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Какая пара находится на втором месте по популярности?

In [74]:
# количество вакансий для каждого сочетания типа рабочего графика типа трудоустройства
query_vac_are = f'''select 
                        concat(schedule, '__', employment) тип_рабочего_графика_и_тип_трудоустройства,
                        count(id) cnt
            from public.vacancies
            group by concat(schedule, '__', employment)
            order by cnt desc
        '''
# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
vacanc_employer = pd.read_sql_query(query_vac_are, connection)
vacanc_employer.head(5)

,тип_рабочего_графика_и_тип_трудоус,cnt
0,Полный день__Полная занятость,35367
1,Удаленная работа__Полная занятость,7802
2,Гибкий график__Полная занятость,1593
3,Удаленная работа__Частичная занятость,1312
4,Сменный график__Полная занятость,940


БОльше всего работодатели ищут работников с занятьстью только задачами работодателя, без совмещения с другим местом работы и обязательным присутствием на рабочем месте. Хотя те, кто смог построить работу с удалёнкой и экономией на аренде офисного помещения так же идёт с большим отрывом от остальных типов занятости. Будущее за удалёнкой.

Задание 4.5

Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [61]:
# код запроса представляет из себя строковую переменную
query_vac_are = f'''select 
                        experience опыт_работы,
                        count(id) cnt
            from public.vacancies
            group by experience
            order by cnt desc
        '''
# лучше использовать f-строки — так удобнее передавать параметры

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
vacanc_employer = pd.read_sql_query(query_vac_are, connection)
vacanc_employer.head(5)

,опыт_работы,cnt
0,От 1 года до 3 лет,26152
1,От 3 до 6 лет,14511
2,Нет опыта,7197
3,Более 6 лет,1337


Тех, кто хочет иметь работников с малым опытом работы и минимальной зарплатой на которых можно вешать непосильные задачи гораздо больше тех, кто полагается на специалистов в своём деле. 

5. Анализ работодателей

Задание 5.1

Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [8]:
# Поиск топ 5 среди работодателей
query_vac_are = f'''select 
                        e.name работадатель,
                        count(v.id) cnt
            from public.vacancies v
            left join public.employers e on v.employer_id = e.id
            group by e.name
            order by cnt desc
        '''
# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
vacanc_employer = pd.read_sql_query(query_vac_are, connection)
vacanc_employer.head(5)

,работадатель,cnt
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


In [86]:
# Сферы деятельности в топ 5 среди работодателей
query_vac_are = f'''select 
                        e.name работадатель,
                        count(v.id) cnt,
                        i.name сфера_деятельности,
                        count(i.id) cnt_i
            from public.vacancies v
            left join public.employers e on v.employer_id = e.id
            join public.employers_industries ei on ei.employer_id = e.id
            join public.industries i on i.id = ei.industry_id
            group by 1, 3
            order by cnt desc
        '''

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
vacanc_employer = pd.read_sql_query(query_vac_are, connection)
vacanc_employer.head(15)

,работадатель,cnt,сфера_деятельности,cnt_i
0,Яндекс,1933,"Интернет-компания (поисковики, платежные систе...",1933
1,Ростелеком,491,Интернет-провайдер,491
2,Ростелеком,491,Мобильная связь,491
3,Ростелеком,491,Оптоволоконная связь,491
4,Ростелеком,491,Фиксированная связь,491
5,Тинькофф,444,Коллекторская деятельность,444
6,Тинькофф,444,Банк,444
7,Тинькофф,444,"Интернет-компания (поисковики, платежные систе...",444
8,Тинькофф,444,"Системная интеграция, автоматизации технологи...",444
9,Тинькофф,444,Разработка программного обеспечения,444


Среди топ пятёрки работодателей сфера деятельности и искомые специальности не все связаны с IT.

Задание 5.2

Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.

Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей. Впишите его название в поле ниже в том виде, который вернул запрос.

In [58]:
# код запроса представляет из себя строковую переменную
query_vac_are = f'''select 
                        a.name регион,
                        count(e.name) кол_работадателей,
                        count(v.id) кол_вакансий
            from public.employers e 
            left join public.areas a on e.area = a.id
            left join public.vacancies v on v.area_id = a.id
            group by a.name
            having count(v.id) = 0             
            order by кол_работадателей desc, кол_вакансий
        '''
# лучше использовать f-строки — так удобнее передавать параметры

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
vacanc_areas_empl = pd.read_sql_query(query_vac_are, connection)
vacanc_areas_empl.head(1)

,регион,кол_работадателей,кол_вакансий
0,Россия,410,0


Данное исследование говорит о не корректном заполнении базы данных. 

Задание 5.3

Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии.

Выберите максимальное значение из получившегося списка.

In [57]:
# код запроса представляет из себя строковую переменную
query_vac_are = f'''select 
                        e.name работадатель,
                        count(distinct v.area_id) кол_регионов
            from public.employers e
            join public.vacancies v on v.employer_id = e.id
            group by e.name            
            order by кол_регионов desc
        '''
# лучше использовать f-строки — так удобнее передавать параметры

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
vacanc_areas_empl = pd.read_sql_query(query_vac_are, connection)
vacanc_areas_empl.head(1)

,работадатель,кол_регионов
0,Яндекс,181


In [34]:
# Основные типы графика работы для компании Яндекс
query_vac_are = f'''select 
                        e.name ,
                        v.schedule ,
                        count(v.schedule) cnt_sched
            from public.employers e
            join public.vacancies v on v.employer_id = e.id
            where e.name = 'Яндекс'
            group by e.name, v.schedule      
            order by cnt_sched desc
        '''

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
vacanc_areas_empl = pd.read_sql_query(query_vac_are, connection)
vacanc_areas_empl

,name,schedule,cnt_sched
0,Яндекс,Удаленная работа,1690
1,Яндекс,Полный день,131
2,Яндекс,Сменный график,57
3,Яндекс,Гибкий график,55


In [35]:
# Основные типы трудоустройства для компании Яндекс
query_vac_are = f'''select 
                        e.name ,
                        v.employment ,
                        count(v.employment) cnt_empl
            from public.employers e
            join public.vacancies v on v.employer_id = e.id
            where e.name = 'Яндекс'
            group by e.name, v.employment        
            order by cnt_empl desc
        '''

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
vacanc_areas_empl = pd.read_sql_query(query_vac_are, connection)
vacanc_areas_empl

,name,employment,cnt_empl
0,Яндекс,Частичная занятость,1135
1,Яндекс,Полная занятость,786
2,Яндекс,Стажировка,12


Компания Яндекс находится на первом месте среди работодателей, с большим отрывом по вакансиям от ближайших компаний. Соответственно и количество регионов, где она представлена самое большое. Соглано дополнительному исследованию такое большое количество регионов, где представлен Яндекс объясняется и тем, что самое большое количество предложений - это удалённая работа и частичная занятость. В каждом регионе не надо организовывать офис.

Задание 5.4

Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.


In [56]:
# подсчёт количества работодателей, у которых не указана сфера деятельности
query_empl_industr = f'''select
                        count(*) cnt
            from public.employers e
            left join public.employers_industries ei on ei.employer_id = e.id
            where ei.industry_id is Null

        '''
# лучше использовать f-строки — так удобнее передавать параметры

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
vacanc_empl_industr = pd.read_sql_query(query_empl_industr, connection)
vacanc_empl_industr.iloc[0]

cnt    8419
Name: 0, dtype: int64

Большое количество работодателей у которых не указана сфера деятельности скорее связана с тем, что это бюджетные гос организации.

Задание 5.5

Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [68]:
# компании, у которых указано четыре сферы деятельности
query_empl_industr = f'''select 
                        distinct e.name,
                        count(ei.industry_id)
            from public.employers e
            left join public.employers_industries ei on ei.employer_id = e.id
            group by e.name
            having count(ei.industry_id) = 4
            order by e.name
       
        '''

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
vacanc_empl_industr = pd.read_sql_query(query_empl_industr, connection)
vacanc_empl_industr.head(3)

,name,count
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4


Задание 5.6

С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».

In [40]:
# Компании специализирующиеся на разработке программного обеспечения».
query_empl_industr = f'''select 
                        count(i.name)
            from public.employers e
            left join public.employers_industries ei on ei.employer_id = e.id
            left join public.industries i on i.id = ei.industry_id
            where i.name = 'Разработка программного обеспечения'
      
        '''

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
vacanc_empl_industr = pd.read_sql_query(query_empl_industr, connection)
vacanc_empl_industr.iloc[0]

count    3553
Name: 0, dtype: int64

In [39]:
# Компании общее количество
query_empl_industr = f'''select 
                        count(name)
            from public.employers
            '''

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
vacanc_empl_industr = pd.read_sql_query(query_empl_industr, connection)
vacanc_empl_industr.iloc[0]

count    23501
Name: 0, dtype: int64

Всего 15% компаний от общего количества занимается разработкой программного обеспечения.

Задание 5.7

Для компании «Яндекс» выведите список регионов-миллионников , в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании, собранных в этой таблице. Должна получиться выборка такого вида (приведён пример результата для компании SberTech):

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/997f04a95aa6fa168c21c592aa6a69d4/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/SQL_pj2_5_1.png)

In [34]:
# Яндекс в городах миллионниках
query_empl_industr = f'''(select 
                        a.name ,
                        count(v.id)
            from public.vacancies v
            left join public.employers e on v.employer_id = e.id
            join public.areas a on a.id = v.area_id
            where e.name = 'Яндекс' and
            a.name in ('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань',
            'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону',
            'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград')
            group by a.name
            order by a.name)
            
            UNION
            
            select 
                'Total',
                count(*)
            from public.vacancies v
            left join public.employers e on v.employer_id = e.id
            join public.areas a on a.id = v.area_id
            where e.name = 'Яндекс' and
            a.name in ('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань',
            'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону',
            'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград')
            order by count
            
        '''

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
vacanc_empl_industr = pd.read_sql_query(query_empl_industr, connection)
vacanc_empl_industr

,name,count
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


Если не брать во внимание столицы, количество вакансий в городах миллиониках не сильно отличается скорее всего потому, что Яндекс практикует удалённую работу

Задание 6.1

Сколько вакансий имеет отношение к данным в названии 'данн' или 'data'

In [30]:
# Количество вакансий для DS
query_empl_industr = f'''select
                        name vacansy_ds
            from public.vacancies
            where (name ilike '%data%' or name ilike '%данн%')
            order by name
       
        '''

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
vacanc = pd.read_sql_query(query_empl_industr, connection)
vacanc.shape[0]

1771

Анализ данных явно набирает популярность, среди вакансий. Знать и анализировать деятельность кампаний для принятия правильного направления развития - много значит

Задание 6.2

Сколько есть подходящих вакансий для начинающего дата-сайентиста?

Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:

* ‘data scientist’;
* ‘data science’;
* ‘исследователь данных’;
* ‘ML’ (здесь не нужно брать вакансии по HTML);
* ‘machine learning’;
* ‘машинн%обучен%’.
В следующих заданиях мы продолжим работать с вакансиями по указанному выше условию (уже не учитывая вакансии уровня Junior).

Считаем вакансиями для специалистов уровня Junior следующие:

* в названии есть слово “junior” или
* требуемый опыт — «Нет опыта» или
* тип трудоустройства — «Стажировка».

In [29]:
# Количество вакансий для DS Junior
query_empl_industr = f'''select
                        count(name) junior_ds_vacansy

            from public.vacancies
            where ((name ilike '%data scientist%' or name ilike '%data science%' or name ilike '%исследователь данных%')
                or ((name like '%ML%') and (name NOT like '%HTML%'))
                or (name ilike '%machine learning%')
                or (name ilike '%машинн%обучен%')
                and (key_skills is NOT NULL))
                and (name ilike '%junior%'
                or experience like '%Нет опыта%' or employment like '%Стажировка%')

       
        '''

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
vacanc_ds = pd.read_sql_query(query_empl_industr, connection)
vacanc_ds.iloc[0]

junior_ds_vacansy    51
Name: 0, dtype: int64

Всего 2,9% вакансий для начинающих DS. Всем надо опытных аналитиков. От их исследования зависит много. (И где тогда набраться опыта?)

Задание 6.3

Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

In [22]:
# Популярность SQL в DS
query_empl_industr = f'''select
                        name,
                        key_skills
            from public.vacancies
            where ((name ilike '%data scientist%' or name ilike '%data science%' or name ilike '%исследователь данных%')
                or ((name like '%ML%') and (name NOT like '%HTML%'))
                or (name ilike '%machine learning%')
                or (name ilike '%машинн%обучен%')
                and (key_skills is NOT NULL))
                and (key_skills ilike '%sql%' or key_skills ilike '%postgres%')

            order by name
       
        '''

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
vacanc_ds = pd.read_sql_query(query_empl_industr, connection)
vacanc_ds.shape[0]

201

11,3% для спецов по SQL от общего числа вакансий для DS. Как-то маловато. Этот тип аналитики не сильно популярен. Или большое количество спецов уже работающих.

Задание 6.4

С помощью запроса, аналогичного предыдущему, проверьте, насколько популярен Python в требованиях работодателей к DS. Вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

In [21]:
# Популярность Python в DS
query_empl_industr = f'''select
                        name,
                        key_skills
            from public.vacancies
            where ((name ilike '%data scientist%' or name ilike '%data science%' or name ilike '%исследователь данных%')
                or ((name like '%ML%') and (name NOT like '%HTML%'))
                or (name ilike '%machine learning%')
                or (name ilike '%машинн%обучен%')
                and (key_skills is NOT NULL))
                and (key_skills ilike '%python%')

            order by name
       
        '''

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
vacanc_ds = pd.read_sql_query(query_empl_industr, connection)
vacanc_ds.shape[0]

351

Почти 20% вакансий для DS на Python - пятая часть от рынка вакансий. Хороший показатель. Связан с популярность даного языка и его большими возможностями.

Задание 6.5

Сколько ключевых навыков в среднем указывают в вакансиях для DS?

Ответ округлите до двух знаков после точки-разделителя.

In [19]:
# Навыки в DS
query_empl_industr = f'''select
                        round(avg(length(key_skills) - length(replace(key_skills,CHR(9),''))+1), 2) round_skills
            from public.vacancies
            where ((name ilike '%data scientist%' or name ilike '%data science%' or name ilike '%исследователь данных%')
                or ((name like '%ML%') and (name NOT like '%HTML%'))
                or (name ilike '%machine learning%')
                or (name ilike '%машинн%обучен%')
                and (key_skills is NOT NULL))
       
        '''

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
vacanc_ds = pd.read_sql_query(query_empl_industr, connection)
vacanc_ds.iloc[0]

round_skills    6.41
Name: 0, dtype: float64

Дата саентист, судя по количеству предъявляемых навыков должен быть многопрофильным спецом. Хотя знать всё и везде хорошо разбираться - просто не реально.

Задание 6.6

Напишите запрос, позволяющий вычислить, какую зарплату для DS в среднем указывают для каждого типа требуемого опыта (уникальное значение из поля experience).

При решении задачи примите во внимание следующее:

* Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
* Если заполнены оба поля с зарплатой, считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. 
* Если заполнено только одно из полей, его и считаем зарплатой по вакансии.
* Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null).
* Чтобы избежать этой ситуации, мы воспользуемся функцией coalesce , которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос select 1 + coalesce(null, 0).
* Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. * * * Результат округлите до целого числа.

In [40]:
# Уровни средней зарплаты в DS
query_empl_industr = f'''SELECT 
                        experience,
                        ROUND(AVG(coalesce((vacancies.salary_from + vacancies.salary_to) / 2, 
                        vacancies.salary_from, vacancies.salary_to)), 0) salary
                    FROM vacancies   
                    where ((name ilike '%data scientist%' or name ilike '%data science%' or name ilike '%исследователь данных%')
                        or ((name like '%ML%') and (name NOT like '%HTML%'))
                        or (name ilike '%machine learning%')
                        or (name ilike '%машинн%обучен%')
                        and (key_skills is NOT NULL))
    
                    GROUP BY vacancies.experience
       
        '''

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
vacanc_salary = pd.read_sql_query(query_empl_industr, connection)
vacanc_salary

,experience,salary
0,Более 6 лет,NaN
1,Нет опыта,74643.0
2,От 1 года до 3 лет,145236.0
3,От 3 до 6 лет,243115.0


Опыт даёт хорошую возможность хорошо зарабатывать. Между градациями опыта почти двойное различие уровней зарплаты.И полное отсутствие с опытом от 6 лет. Возможно это связано с новизной этого направления IT специальности.